In [1]:
import pandas as pd
from aequitas.group import Group
from aequitas.bias import Bias
from aequitas.fairness import Fairness
import aequitas.plot as ap
import numpy as np

# Enable Pandas to display dataframes without restriction.
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [29]:
df = pd.read_csv("bias_analysis_data.csv")
df.head(10)

,Unnamed: 0,age,workclass,fnlgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary,label_value,score
0,0,53,Self-emp-not-inc,168539,9th,5,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,70,United-States,<=50K,0,0
1,1,49,Self-emp-not-inc,56841,Bachelors,13,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,70,United-States,<=50K,0,0
2,2,28,Private,154571,Some-college,10,Married-civ-spouse,Exec-managerial,Husband,Asian-Pac-Islander,Male,0,0,40,Philippines,<=50K,0,0
3,3,60,Private,188236,10th,6,Widowed,Tech-support,Unmarried,White,Female,0,0,40,United-States,<=50K,0,0
4,4,53,Private,87158,HS-grad,9,Married-civ-spouse,Craft-repair,Husband,Black,Male,0,0,40,United-States,<=50K,0,0
5,5,55,Private,125000,Masters,14,Divorced,Exec-managerial,Unmarried,White,Male,0,0,40,United-States,>50K,1,1
6,6,27,Private,376936,HS-grad,9,Never-married,Craft-repair,Not-in-family,White,Male,0,0,40,United-States,<=50K,0,0
7,7,31,Private,163303,Assoc-voc,11,Divorced,Sales,Own-child,White,Female,0,0,38,United-States,<=50K,0,0
8,8,30,Private,112650,HS-grad,9,Divorced,Craft-repair,Own-child,White,Male,0,0,40,United-States,<=50K,0,0
9,9,57,Federal-gov,414994,Some-college,10,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,40,United-States,>50K,1,1


In [30]:
str_cols = ["race", "sex", "native-country", "score", "label_value"]
df = df.astype({col:"str" for col in df.columns if col not in ["score", "label_value"]})
df = df[str_cols]

In [31]:
group = Group()
xtab, _ = group.get_crosstabs(df)

xtab.head(10)

,model_id,score_threshold,k,attribute_name,attribute_value,tpr,tnr,for,fdr,fpr,fnr,npv,precision,pp,pn,ppr,pprev,fp,fn,tn,tp,group_label_pos,group_label_neg,group_size,total_entities,prev
0,0,binary 0/1,5976,race,Amer-Indian-Eskimo,1.000000,1.0,0.000000,0.0,0.0,0.000000,1.000000,1.0,28,204,0.004685,0.120690,0,0,204,28,28,204,232,24129,0.120690
1,0,binary 0/1,5976,race,Asian-Pac-Islander,1.000000,1.0,0.000000,0.0,0.0,0.000000,1.000000,1.0,197,511,0.032965,0.278249,0,0,511,197,197,511,708,24129,0.278249
2,0,binary 0/1,5976,race,Black,1.000000,1.0,0.000000,0.0,0.0,0.000000,1.000000,1.0,290,1951,0.048527,0.129407,0,0,1951,290,290,1951,2241,24129,0.129407
3,0,binary 0/1,5976,race,Other,1.000000,1.0,0.000000,0.0,0.0,0.000000,1.000000,1.0,15,176,0.002510,0.078534,0,0,176,15,15,176,191,24129,0.078534
4,0,binary 0/1,5976,race,White,0.999633,1.0,0.000131,0.0,0.0,0.000367,0.999869,1.0,5446,15311,0.911312,0.262369,0,2,15309,5446,5448,15309,20757,24129,0.262466
5,0,binary 0/1,5976,sex,Female,0.998876,1.0,0.000144,0.0,0.0,0.001124,0.999856,1.0,889,6932,0.148762,0.113668,0,1,6931,889,890,6931,7821,24129,0.113796
6,0,binary 0/1,5976,sex,Male,0.999803,1.0,0.000089,0.0,0.0,0.000197,0.999911,1.0,5087,11221,0.851238,0.311933,0,1,11220,5087,5088,11220,16308,24129,0.311994
7,0,binary 0/1,5976,native-country,Cambodia,1.000000,1.0,0.000000,0.0,0.0,0.000000,1.000000,1.0,4,11,0.000669,0.266667,0,0,11,4,4,11,15,24129,0.266667
8,0,binary 0/1,5976,native-country,Canada,1.000000,1.0,0.000000,0.0,0.0,0.000000,1.000000,1.0,30,58,0.005020,0.340909,0,0,58,30,30,58,88,24129,0.340909
9,0,binary 0/1,5976,native-country,China,1.000000,1.0,0.000000,0.0,0.0,0.000000,1.000000,1.0,19,37,0.003179,0.339286,0,0,37,19,19,37,56,24129,0.339286


In [32]:
bias_df = bias.get_disparity_major_group(xtab,
                               original_df=df,
                               alpha=0.05,
                               mask_significance=True).head(10)
bias_df.head(10)

get_disparity_major_group()


,model_id,score_threshold,k,attribute_name,attribute_value,tpr,tnr,for,fdr,fpr,fnr,npv,precision,pp,pn,ppr,pprev,fp,fn,tn,tp,group_label_pos,group_label_neg,group_size,total_entities,prev,ppr_disparity,pprev_disparity,precision_disparity,fdr_disparity,for_disparity,fpr_disparity,fnr_disparity,tpr_disparity,tnr_disparity,npv_disparity,ppr_ref_group_value,pprev_ref_group_value,precision_ref_group_value,fdr_ref_group_value,for_ref_group_value,fpr_ref_group_value,fnr_ref_group_value,tpr_ref_group_value,tnr_ref_group_value,npv_ref_group_value
0,0,binary 0/1,5976,race,Amer-Indian-Eskimo,1.000000,1.0,0.000000,0.0,0.0,0.000000,1.000000,1.0,28,204,0.004685,0.120690,0,0,204,28,28,204,232,24129,0.120690,0.005141,0.459999,1.0,NaN,0.000000,NaN,0.000000,1.000367,1.0,1.000131,White,White,White,White,White,White,White,White,White,White
1,0,binary 0/1,5976,race,Asian-Pac-Islander,1.000000,1.0,0.000000,0.0,0.0,0.000000,1.000000,1.0,197,511,0.032965,0.278249,0,0,511,197,197,511,708,24129,0.278249,0.036173,1.060523,1.0,NaN,0.000000,NaN,0.000000,1.000367,1.0,1.000131,White,White,White,White,White,White,White,White,White,White
2,0,binary 0/1,5976,race,Black,1.000000,1.0,0.000000,0.0,0.0,0.000000,1.000000,1.0,290,1951,0.048527,0.129407,0,0,1951,290,290,1951,2241,24129,0.129407,0.053250,0.493223,1.0,NaN,0.000000,NaN,0.000000,1.000367,1.0,1.000131,White,White,White,White,White,White,White,White,White,White
3,0,binary 0/1,5976,race,Other,1.000000,1.0,0.000000,0.0,0.0,0.000000,1.000000,1.0,15,176,0.002510,0.078534,0,0,176,15,15,176,191,24129,0.078534,0.002754,0.299326,1.0,NaN,0.000000,NaN,0.000000,1.000367,1.0,1.000131,White,White,White,White,White,White,White,White,White,White
4,0,binary 0/1,5976,race,White,0.999633,1.0,0.000131,0.0,0.0,0.000367,0.999869,1.0,5446,15311,0.911312,0.262369,0,2,15309,5446,5448,15309,20757,24129,0.262466,1.000000,1.000000,1.0,NaN,1.000000,NaN,1.000000,1.000000,1.0,1.000000,White,White,White,White,White,White,White,White,White,White
5,0,binary 0/1,5976,sex,Female,0.998876,1.0,0.000144,0.0,0.0,0.001124,0.999856,1.0,889,6932,0.148762,0.113668,0,1,6931,889,890,6931,7821,24129,0.113796,0.174759,0.364400,1.0,NaN,1.618725,NaN,5.716854,0.999073,1.0,0.999945,Male,Male,Male,Male,Male,Male,Male,Male,Male,Male
6,0,binary 0/1,5976,sex,Male,0.999803,1.0,0.000089,0.0,0.0,0.000197,0.999911,1.0,5087,11221,0.851238,0.311933,0,1,11220,5087,5088,11220,16308,24129,0.311994,1.000000,1.000000,1.0,NaN,1.000000,NaN,1.000000,1.000000,1.0,1.000000,Male,Male,Male,Male,Male,Male,Male,Male,Male,Male
7,0,binary 0/1,5976,native-country,Cambodia,1.000000,1.0,0.000000,0.0,0.0,0.000000,1.000000,1.0,4,11,0.000669,0.266667,0,0,11,4,4,11,15,24129,0.266667,0.000719,1.053628,1.0,NaN,0.000000,NaN,0.000000,1.000360,1.0,1.000122,United-States,United-States,United-States,United-States,United-States,United-States,United-States,United-States,United-States,United-States
8,0,binary 0/1,5976,native-country,Canada,1.000000,1.0,0.000000,0.0,0.0,0.000000,1.000000,1.0,30,58,0.005020,0.340909,0,0,58,30,30,58,88,24129,0.340909,0.005393,1.346968,1.0,NaN,0.000000,NaN,0.000000,1.000360,1.0,1.000122,United-States,United-States,United-States,United-States,United-States,United-States,United-States,United-States,United-States,United-States
9,0,binary 0/1,5976,native-country,China,1.000000,1.0,0.000000,0.0,0.0,0.000000,1.000000,1.0,19,37,0.003179,0.339286,0,0,37,19,19,37,56,24129,0.339286,0.003415,1.340554,1.0,NaN,0.000000,NaN,0.000000,1.000360,1.0,1.000122,United-States,United-States,United-States,United-States,United-States,United-States,United-States,United-States,United-States,United-States


In [33]:
fairness = Fairness()
fairness_df = fairness.get_group_value_fairness(bias_df)
fairness_df.head(10)

,model_id,score_threshold,k,attribute_name,attribute_value,tpr,tnr,for,fdr,fpr,fnr,npv,precision,pp,pn,ppr,pprev,fp,fn,tn,tp,group_label_pos,group_label_neg,group_size,total_entities,prev,ppr_disparity,pprev_disparity,precision_disparity,fdr_disparity,for_disparity,fpr_disparity,fnr_disparity,tpr_disparity,tnr_disparity,npv_disparity,ppr_ref_group_value,pprev_ref_group_value,precision_ref_group_value,fdr_ref_group_value,for_ref_group_value,fpr_ref_group_value,fnr_ref_group_value,tpr_ref_group_value,tnr_ref_group_value,npv_ref_group_value,Statistical Parity,Impact Parity,FDR Parity,FPR Parity,FOR Parity,FNR Parity,TPR Parity,TNR Parity,NPV Parity,Precision Parity,TypeI Parity,TypeII Parity,Equalized Odds,Unsupervised Fairness,Supervised Fairness
0,0,binary 0/1,5976,race,Amer-Indian-Eskimo,1.000000,1.0,0.000000,0.0,0.0,0.000000,1.000000,1.0,28,204,0.004685,0.120690,0,0,204,28,28,204,232,24129,0.120690,0.005141,0.459999,1.0,NaN,0.000000,NaN,0.000000,1.000367,1.0,1.000131,White,White,White,White,White,White,White,White,White,White,False,False,NaN,NaN,False,False,True,True,True,True,NaN,False,False,False,False
1,0,binary 0/1,5976,race,Asian-Pac-Islander,1.000000,1.0,0.000000,0.0,0.0,0.000000,1.000000,1.0,197,511,0.032965,0.278249,0,0,511,197,197,511,708,24129,0.278249,0.036173,1.060523,1.0,NaN,0.000000,NaN,0.000000,1.000367,1.0,1.000131,White,White,White,White,White,White,White,White,White,White,False,True,NaN,NaN,False,False,True,True,True,True,NaN,False,False,False,False
2,0,binary 0/1,5976,race,Black,1.000000,1.0,0.000000,0.0,0.0,0.000000,1.000000,1.0,290,1951,0.048527,0.129407,0,0,1951,290,290,1951,2241,24129,0.129407,0.053250,0.493223,1.0,NaN,0.000000,NaN,0.000000,1.000367,1.0,1.000131,White,White,White,White,White,White,White,White,White,White,False,False,NaN,NaN,False,False,True,True,True,True,NaN,False,False,False,False
3,0,binary 0/1,5976,race,Other,1.000000,1.0,0.000000,0.0,0.0,0.000000,1.000000,1.0,15,176,0.002510,0.078534,0,0,176,15,15,176,191,24129,0.078534,0.002754,0.299326,1.0,NaN,0.000000,NaN,0.000000,1.000367,1.0,1.000131,White,White,White,White,White,White,White,White,White,White,False,False,NaN,NaN,False,False,True,True,True,True,NaN,False,False,False,False
4,0,binary 0/1,5976,race,White,0.999633,1.0,0.000131,0.0,0.0,0.000367,0.999869,1.0,5446,15311,0.911312,0.262369,0,2,15309,5446,5448,15309,20757,24129,0.262466,1.000000,1.000000,1.0,NaN,1.000000,NaN,1.000000,1.000000,1.0,1.000000,White,White,White,White,White,White,White,White,White,White,True,True,NaN,NaN,True,True,True,True,True,True,NaN,True,False,True,False
5,0,binary 0/1,5976,sex,Female,0.998876,1.0,0.000144,0.0,0.0,0.001124,0.999856,1.0,889,6932,0.148762,0.113668,0,1,6931,889,890,6931,7821,24129,0.113796,0.174759,0.364400,1.0,NaN,1.618725,NaN,5.716854,0.999073,1.0,0.999945,Male,Male,Male,Male,Male,Male,Male,Male,Male,Male,False,False,NaN,NaN,False,False,True,True,True,True,NaN,False,False,False,False
6,0,binary 0/1,5976,sex,Male,0.999803,1.0,0.000089,0.0,0.0,0.000197,0.999911,1.0,5087,11221,0.851238,0.311933,0,1,11220,5087,5088,11220,16308,24129,0.311994,1.000000,1.000000,1.0,NaN,1.000000,NaN,1.000000,1.000000,1.0,1.000000,Male,Male,Male,Male,Male,Male,Male,Male,Male,Male,True,True,NaN,NaN,True,True,True,True,True,True,NaN,True,False,True,False
7,0,binary 0/1,5976,native-country,Cambodia,1.000000,1.0,0.000000,0.0,0.0,0.000000,1.000000,1.0,4,11,0.000669,0.266667,0,0,11,4,4,11,15,24129,0.266667,0.000719,1.053628,1.0,NaN,0.000000,NaN,0.000000,1.000360,1.0,1.000122,United-States,United-States,United-States,United-States,United-States,United-States,United-States,United-States,United-States,United-States,False,True,NaN,NaN,False,False,True,True,True,True,NaN,False,False,False,False
8,0,binary 0/1,5976,native-country,Canada,1.000000,1.0,0.000000,0.0,0.0,0.000000,1.000000,1.0,30,58,0.005020,0.340909,0,0,58,30,30,58,88,24129,0.340909,0.005393,1.346968,1.0,NaN,0.000000,NaN,0.000000,1.000360,1.0,1.000122,United-States,United-States,United-States,United-States,United

In [34]:
overall_fairness = fairness.get_overall_fairness(fairness_df)
print(overall_fairness)

{'Unsupervised Fairness': False, 'Supervised Fairness': False, 'Overall Fairness': False}
